In [ ]:
from setproctitle import setproctitle

setproctitle("nf2")

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
nf2_file = "/mnt/obsdata/nf2/notebook/7115_series/20170906_000000_TAI.nf2"

In [ ]:
ext = os.path.basename(nf2_file).split(".")[1]

In [ ]:
os.path.basename(nf2_file).split(".")[0][:-4].replace("_", "T")

'20170906T000000'

In [ ]:
from tool.nf2.evaluation.unpack import load_cube
from tool.nf2.potential.potential_field import get_potential_field

In [ ]:
B = load_cube(nf2_file, progress=True)
B_pot = get_potential_field(B[:, :, 0, 2], B.shape[2], batch_size=int(1e3))

Potential Field: 100%|██████████| 12329/12329 [02:56<00:00, 69.91it/s]


In [ ]:
B.shape

(344, 224, 160, 3)

In [ ]:
B_pot.shape

(344, 224, 160, 3)

In [ ]:
Bx = B[..., 0]
By = B[..., 1]
Bz = B[..., 2]

Bx_pot = B_pot[..., 0]
By_pot = B_pot[..., 1]
Bz_pot = B_pot[..., 2]

In [ ]:
import numpy as np
import torch

In [ ]:
state = torch.load(nf2_file)

In [ ]:
state

{'model': BModel(
   (d_in): Linear(in_features=3, out_features=256, bias=True)
   (linear_layers): ModuleList(
     (0-7): 8 x Linear(in_features=256, out_features=256, bias=True)
   )
   (d_out): Linear(in_features=256, out_features=3, bias=True)
   (activation): Sine()
 ),
 'cube_shape': [344, 224, 160],
 'b_norm': 2500,
 'spatial_norm': 160,
 'meta_data': MetaDict([('simple', True),
           ('bitpix', 32),
           ('naxis', 2),
           ('naxis1', 688),
           ('naxis2', 448),
           ('blank', -2147483648),
           ('bzero', 0.0),
           ('bscale', 0.01),
           ('checksum', 'ZA8ZZ66YZA6YZ56Y'),
           ('datasum', '695910676'),
           ('date', '2017-10-11T03:08:26.000'),
           ('date_s', '2017-09-10T16:27:19.000'),
           ('date_b', '2017-09-12T08:09:18.000'),
           ('date-obs', '2017-09-05T23:58:42.000'),
           ('t_obs', '2017.09.06_00:00:03.957_TAI'),
           ('t_rec', '2017.09.06_00:00:00.000_TAI'),
           ('trecepoc',

In [ ]:
Mm_per_pixel = state["Mm_per_pixel"]
Mm_per_pixel

0.72

In [ ]:
Nx, Ny, Nz = state["cube_shape"]
Nx, Ny, Nz

(344, 224, 160)

In [ ]:
Lx = (Nx - 1) * Mm_per_pixel
Ly = (Ny - 1) * Mm_per_pixel
Lz = (Nz - 1) * Mm_per_pixel

Lx, Ly, Lz

(246.95999999999998, 160.56, 114.47999999999999)

In [ ]:
x = np.linspace(0, Lx, Nx)
x[-1] - x[0]

246.95999999999998

In [ ]:
y = np.linspace(0, Ly, Ny)
z = np.linspace(0, Lz, Nz)

In [ ]:
from tool.evaluate import curl, divergence, laplacian

In [ ]:
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]

dx, dy, dz

(0.72, 0.72, 0.72)

In [ ]:
from tool.metric import calculate_derivative, calculate_metric, draw_projection

In [ ]:
(
    B,
    norm_B,
    J,
    norm_J,
    JxB,
    norm_JxB,
    div_B,
    norm_div_B,
    laplacian_B,
    norm_laplacian_B,
    energy_density_B,
) = calculate_derivative(Bx, By, Bz, dx, dy, dz)

In [ ]:
(
    B_pot,
    norm_B_pot,
    J_pot,
    norm_J_pot,
    JxB_pot,
    norm_JxB_pot,
    div_B_pot,
    norm_div_B_pot,
    laplacian_B_pot,
    norm_laplacian_B_pot,
    energy_density_B_pot,
) = calculate_derivative(Bx_pot, By_pot, Bz_pot, dx, dy, dz)

In [ ]:
dV_cm = (dx * 1e8) * (dy * 1e8) * (dz * 1e8)  # cm^3

free_energy_density = energy_density_B - energy_density_B_pot
total_free_energy = free_energy_density.sum() * dV_cm  # erg

(
    total_energy,
    loss_force_free,
    loss_force_free_mean,
    loss_div_free,
    loss_div_free_mean,
    sigma_J,
    theta_J,
    theta_i_mean,
    norm_laplacian_B_mean,
    max_idx_0,
    norm_laplacian_B_max_0,
    max_idx_1,
    norm_laplacian_B_max_1,
    max_idx_2,
    norm_laplacian_B_max_2,
) = calculate_metric(
    dx,
    dy,
    dz,
    Lx,
    Ly,
    Lz,
    B,
    norm_B,
    J,
    norm_J,
    JxB,
    norm_JxB,
    div_B,
    norm_div_B,
    laplacian_B,
    norm_laplacian_B,
    energy_density_B,
)

(
    total_energy_pot,
    loss_force_free_pot,
    loss_force_free_mean_pot,
    loss_div_free_pot,
    loss_div_free_mean_pot,
    sigma_J_pot,
    theta_J_pot,
    theta_i_mean_pot,
    norm_laplacian_B_mean_pot,
    max_idx_0_pot,
    norm_laplacian_B_max_0_pot,
    max_idx_1_pot,
    norm_laplacian_B_max_1_pot,
    max_idx_2_pot,
    norm_laplacian_B_max_2_pot,
) = calculate_metric(
    dx,
    dy,
    dz,
    Lx,
    Ly,
    Lz,
    B_pot,
    norm_B_pot,
    J_pot,
    norm_J_pot,
    JxB_pot,
    norm_JxB_pot,
    div_B_pot,
    norm_div_B_pot,
    laplacian_B_pot,
    norm_laplacian_B_pot,
    energy_density_B_pot,
)

/userhome/jeon_mg/workspace/codes/Magnetic_Field_Evaluation/tool/metric.py:108: RuntimeWarning: invalid value encountered in arcsin
  theta_i = np.arcsin(sigma_i)


In [ ]:
draw_projection(
    free_energy_density,
    "Free energy density (erg/cm^3)",
    os.path.join("./", "energy_free.png"),
    dx,
    dy,
    dz,
    Lx,
    Ly,
    Lz,
    40,
    cm=True,
    log=False,
    cmap="jet",
)

In [ ]:
draw_projection(
    norm_J,
    r"$|\nabla \times \mathbf{B}|$" + " (G/Mm)",
    os.path.join("./", "J.png"),
    dx,
    dy,
    dz,
    Lx,
    Ly,
    Lz,
    40,
)
draw_projection(
    norm_laplacian_B,
    r"$|\nabla^2 \mathbf{B}|$" + " (G/Mm^2)",
    os.path.join("./", "laplacian_B.png"),
    dx,
    dy,
    dz,
    Lx,
    Ly,
    Lz,
    40,
)

In [ ]:
sigma_J * 100

21.067844331264496